# 2. Activity Prediction

First, we download and extract the data.
We read them into two lists, one containing the input data $\mathcal X$ and one the target classes $\mathbf T$ as integral values.
The input data $\mathcal X \in \mathbb R^{M\times N\times D}$ is organized such that we have $M=15$ subjects with various numbers $N$ of samples, each with a data dimension of $D=3$.
The target data $\mathbf T \in \mathbb \{0,...,6\}^{M\times N}$ provides the target class for each sample.

Please do not modify this code.

In [34]:
import os

# download data
if not os.path.exists("UserActivity.zip"):
  import urllib.request
  urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00287/Activity%20Recognition%20from%20Single%20Chest-Mounted%20Accelerometer.zip", "UserActivity.zip")
  print ("Downloaded datafile", "UserActivity.zip")

# extract data
if not os.path.exists("Activity Recognition from Single Chest-Mounted Accelerometer"):
  import zipfile
  zipfile.ZipFile("UserActivity.zip").extractall()
  print ("Extracted datafile", "UserActivity.zip")

# read the data from the files
X = []
T = []
for i in range(1,16):
  # collect samples for each of the 15 subjects
  with open(f"Activity Recognition from Single Chest-Mounted Accelerometer/{i}.csv") as f:
    # inputs and targets for this subject
    x, t = [], []
    for i, line in enumerate(f):
      # split the data
      splits = line.rstrip().split(",")
      # get the target value
      tt = int(splits[-1])
      # there are some invalid target values, we skip these
      if tt: 
        # load the three accelorometer data
        x.append([float(v) for v in splits[1:-1]])
        # add the label (convert from one-based into zero-based indexing)
        t.append(tt-1)
    # append samples and targets of the current subject
    X.append(x)
    T.append(t)

# print some statistics of the dataset
print (f"number of subjects: {len(X)}\nnumber of samples for the first subject: {len(X[0])}\nlength of one input sample: {len(X[0][0])}")

number of subjects: 15
number of samples for the first subject: 162500
length of one input sample: 3


### 2. (d) Data Reduction

Implement a strategy to reduce the amount of data for each subject. Assure that you apply the identical selection strategy for the inputs and the targets. Make sure that you do not change the arrangement of the data matrices.

In [35]:
import numpy as np

In [43]:
# reduce the number of samples on first dimension by a factor of 5 (every 5th sample)

X_selected = []
T_selected = []
for subject in X:
    subject_data = []
    for i,x in enumerate(subject):
        if i % 2 != 0:
            subject_data.append(x)
    X_selected.append(subject_data)         

for subject in T:
    subject_data = []
    for i,x in enumerate(subject):
        if i % 2 != 0:
            subject_data.append(x)
    T_selected.append(subject_data)       
                

print (f"number of subjects: {len(X_selected)}\nnumber of samples for the first subject: {len(X_selected[0])}\nlength of one input sample: {len(X_selected[0][0])}")
print (f"number of subjects: {len(T_selected)}\nnumber of targets for the first subject: {len(T_selected[0])}")

number of subjects: 15
number of samples for the first subject: 81250
length of one input sample: 3
number of subjects: 15
number of targets for the first subject: 81250


Here, we concatenate the data of subjects 1-10 into the training matrices, and the data of subjects 11-15 into the validation set. 
There is no need to change this code (unless you changed the arrangement of the data above, in which case you also need to adapt this code).

In [44]:
import torch
device = torch.device("cuda")

X_train = torch.tensor(sum(X_selected[:10], []))
X_val = torch.tensor(sum(X_selected[10:], []))

T_train = torch.tensor(sum(T_selected[:10], []))
T_val = torch.tensor(sum(T_selected[10:], []))

print(f"The shape of the training input is: {X_train.shape}\nthe shape of the training targets is {T_train.shape}")

The shape of the training input is: torch.Size([708387, 3])
the shape of the training targets is torch.Size([708387])


### 2. (e) Dataset Implementation

Implement the dataset that takes the given data `X` and `T`, as well as a sequence length `S`. Return a sequence of `S` samples and the label for the last element in the sequence.

In [50]:
class Dataset(torch.utils.data.Dataset):
  
  def __init__(self, X, T, S):
    # implement the constructor
    self.S = S
    self.T = T
    self.X = X

  def __len__(self):
    # return the number of samples in this dataset
    return len(x) #because we want the dimension N, which is the second, i.e. index 1

  def __getitem__(self, index):
    # return the pair of input and target values for the given index
    return self.X[index], self.T[index]

### 2. (f) Data Loaders

We need to instantiate training and validation set data loaders.

In [51]:
# instantiate training set data loader
train_set = Dataset(X_train, T_train, len(X_selected[0]))
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

# and the validation set data loader
val_set = Dataset(X_val, T_val, len(X_selected[0]))
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=False)

TypeError: object of type 'int' has no len()

### 2. (g) Network Implementation

Implement and instantiate one of the networks discussed in (a). Since there are various different ways to implement this network, no guidelines will be provided here. Note that the network should output the prediction only for the last sequence element.

In [ ]:
# implement and instantiate the network

class ElmanNetwork(torch.nn.Module):
  def __init__(self, D, K):
    super(ElmanNetwork,self).__init__()
    self.W1 = torch.nn.Linear(D, K)
    self.Wr = torch.nn.Linear(K, K)
    self.W2 = torch.nn.Linear(K, D)
    self.activation = torch.nn.Softmax()
    self.K = K

  def forward(self, x):
    # get the shape of the data
    B, S, D = x.shape
    # initialize the hidden vector in the desired size with 0
    # remember to put it on the device
    h_s = torch.zeros((B,self.K)).to(device)
    # store all logits (we will need them in the loss function)
    Z = torch.empty(x.shape, device=device)
    # iterate over the sequence
    for s in range(S):
      # use current sequence item and all batches simultaneously (:) (shape = B,S,D) -> the D sample are taken in individually (x)
      x_s = x[:, s]
      # compute recurrent activation: h_s is still stored from the last iteration of S and h[0]=0
      a_s = self.W1(x_s) + self.Wr(h_s)
      # apply activation function
      h_s = self.activation(a_s)
      # compute logit values
      z = self.W2(h_s)
      # store logit values for sequence element s and all batches
      Z[:,s] = z

    # return logits for all sequence elements
    return Z

network = ...

### 2. (h) Network Training

Instantiate the loss function and the optimizer. Train the network for 10 epochs and compute validation set accuracy. Note that one epoch of training might take several minutes. There is no need to wait for the results.

In [ ]:
# instantiate optimizer and loss function
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=network.parameters(),lr=0.01, momentum=0.9) #could also use Adam optimiser

val_acc = []

# train the network for 10 epochs
for epoch in range(10):
  # use all training samples in batches
   for x, t in train_loader:
    x = x.to(device)
    t = t.to(device)
    optimizer.zero_grad()
    # compute network output
    z = network(x)
    # compute loss, arrange order of logits and targets
    #change order of dimensions for x, because the loss is computed over D,
    #which is the third dimension, but needs to be the second one
    J = loss(z.permute(0,2,1),t.permute(0,2,1))
    # compute gradient for this batch
    J.backward()
    optimizer.step()
    # select a new sequence length S in [5,20]
    epoch_length = np.random.randint(5,20)    

  # compute validation set accuracy
   with torch.no_grad():
      cur_acc = 0. #for each epoch, we compute the validation acc
      for x,t in val_loader:
            z = network(x.to(device))
            j = loss(z, t.to(device))
            cur_acc += torch.sum(torch.argmax(z, dim=1) == t.to(device).item()) #accuracy identity function (last time the accuracy function was implemented seperately)
      val_acc.append(cur_acc/(len(testset)))

  # report validation set accuracy
  print(f"epoch {epoch+1}, validation accuracy: {...}")